# Engine to Geo-code Transit Boarding Location to Travel Model Two Transit Access Points

## Purpose
Use cartesian geometry to find the nearest Travel Model Two transit access point, segmented by technology, for each observed on-board survey record.  Locate the first transit boarding TAP and the last transit alighting TAP.

## Notes
Transit boarding locations outside the 9-county Bay Area (e.g., ACE stations in San Joaquin) will be located to the nearest 9-county Bay Area transit location.


#### Needed libraries

In [1]:
import datetime
import pandas
import numpy as np
from numpy import random,argsort,sqrt

#### TAP Version 0.0
Note the TAPs were re-projected to Lat/Long

In [2]:
filename_taps = 'M:/Data/OnBoard/Data and Reports/_geocoding Standardized/TAPs/taps_lat_long.csv'
df_taps       = pandas.read_csv(filename_taps, index_col = 'N')
df_taps       = df_taps.drop(['OBJECTID','X','Y','COUNTY','TYPE','ID'],1)

#### Read in the observations and create fields for nearest TAP and distance to nearest TAP

In [3]:
# Boarding
filename_obs_board = 'M:/Data/OnBoard/Data and Reports/_geocoding Standardized/boarding_places_to_be_geocoded.csv'
df_obs_board       = pandas.read_csv(filename_obs_board, index_col = 'Unique_ID')

df_obs_board['tap'] = -999
df_obs_board['tap_dist'] = -999.99999

# Alighting
filename_obs_alight = 'M:/Data/OnBoard/Data and Reports/_geocoding Standardized/alighting_places_to_be_geocoded.csv'
df_obs_alight       = pandas.read_csv(filename_obs_alight, index_col = 'Unique_ID')

df_obs_alight['tap'] = -999
df_obs_alight['tap_dist'] = -999.99999

#### Create a separate dataframe for each mode category, save in a tuple

In [4]:
df_obs_board.first_board_tech.value_counts()

local bus        47049
heavy rail       45050
light rail       18663
commuter rail     5160
express bus        571
ferry               61
Name: first_board_tech, dtype: int64

In [5]:
df_obs_alight.last_alight_tech.value_counts()

local bus        47141
heavy rail       44253
light rail       19065
commuter rail     5286
express bus        500
ferry               50
Name: last_alight_tech, dtype: int64

In [6]:
df_taps.MODE.value_counts()

1    5098
2     793
4     184
5      44
6      42
3      11
Name: MODE, dtype: int64

In [7]:
df_obs_board_local    = df_obs_board[df_obs_board.first_board_tech == 'local bus']
df_obs_board_express  = df_obs_board[df_obs_board.first_board_tech == 'express bus']
df_obs_board_light    = df_obs_board[df_obs_board.first_board_tech == 'light rail']
df_obs_board_ferry    = df_obs_board[df_obs_board.first_board_tech == 'ferry']
df_obs_board_heavy    = df_obs_board[df_obs_board.first_board_tech == 'heavy rail']
df_obs_board_commuter = df_obs_board[df_obs_board.first_board_tech == 'commuter rail']


df_obs_alight_local    = df_obs_alight[df_obs_alight.last_alight_tech == 'local bus']
df_obs_alight_express  = df_obs_alight[df_obs_alight.last_alight_tech == 'express bus']
df_obs_alight_light    = df_obs_alight[df_obs_alight.last_alight_tech == 'light rail']
df_obs_alight_ferry    = df_obs_alight[df_obs_alight.last_alight_tech == 'ferry']
df_obs_alight_heavy    = df_obs_alight[df_obs_alight.last_alight_tech == 'heavy rail']
df_obs_alight_commuter = df_obs_alight[df_obs_alight.last_alight_tech == 'commuter rail']

df_taps_local   = df_taps[df_taps.MODE == 1]
df_taps_express = df_taps[df_taps.MODE == 2]
df_taps_ferry   = df_taps[df_taps.MODE == 3]
df_taps_light   = df_taps[df_taps.MODE == 4]
df_taps_heavy   = df_taps[df_taps.MODE == 5]
df_taps_cmtr    = df_taps[df_taps.MODE == 6]

obs_and_tap_pairs = (['local',    'first_board_lon', 'first_board_lat', df_obs_board_local,     df_taps_local],
                     ['express',  'first_board_lon', 'first_board_lat', df_obs_board_express,   df_taps_express],
                     ['light',    'first_board_lon', 'first_board_lat', df_obs_board_light,     df_taps_light],
                     ['ferry',    'first_board_lon', 'first_board_lat', df_obs_board_ferry,     df_taps_ferry],
                     ['heavy',    'first_board_lon', 'first_board_lat', df_obs_board_heavy,     df_taps_heavy],
                     ['commuter', 'first_board_lon', 'first_board_lat', df_obs_board_commuter,  df_taps_cmtr],
                     ['local',    'last_alight_lon', 'last_alight_lat', df_obs_alight_local,    df_taps_local],
                     ['express',  'last_alight_lon', 'last_alight_lat', df_obs_alight_express,  df_taps_express],
                     ['light',    'last_alight_lon', 'last_alight_lat', df_obs_alight_light,    df_taps_light],
                     ['ferry',    'last_alight_lon', 'last_alight_lat', df_obs_alight_ferry,    df_taps_ferry],
                     ['heavy',    'last_alight_lon', 'last_alight_lat', df_obs_alight_heavy,    df_taps_heavy],
                     ['commuter', 'last_alight_lon', 'last_alight_lat', df_obs_alight_commuter, df_taps_cmtr])

In [8]:
print str(datetime.datetime.now()) + '\n'

for obs_and_tap_pair in obs_and_tap_pairs:
    
    x_field = obs_and_tap_pair[1]
    y_field = obs_and_tap_pair[2]
    
    print 'Processing for ' + '{0: >8}'.format(obs_and_tap_pair[0]) + ' with X = ' + x_field + ' and Y = ' + y_field + '...'
    
    df_obs  = obs_and_tap_pair[3]
    df_taps = obs_and_tap_pair[4]

    obs_index_array = df_obs.index
    
    for obs_index in obs_index_array:
        
        obs_x = df_obs.at[obs_index,x_field]
        obs_y = df_obs.at[obs_index,y_field]
        
        # try df_taps[:,('LONG')] etc to see if this is what is triggering the warning
        df_taps['distance'] = sqrt((obs_x - df_taps['LONG'])**2 + (obs_y - df_taps['LAT'])**2)
    
        nearest_tap          = df_taps['distance'].idxmin(1)
        nearest_tap_distance = df_taps.min()['distance']
    
        df_obs['tap'][obs_index]      = nearest_tap
        df_obs['tap_dist'][obs_index] = nearest_tap_distance
    
print '\n' + str(datetime.datetime.now())

2017-12-27 08:19:45.448000

Processing for    local with X = first_board_lon and Y = first_board_lat...


c:\python27\arcgis10.5\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\python27\arcgis10.5\lib\site-packages\IPython\core\interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
c:\python27\arcgis10.5\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Processing for  express with X = first_board_lon and Y = first_board_lat...
Processing for    light with X = first_board_lon and Y = first_board_lat...
Processing for    ferry with X = first_board_lon and Y = first_board_lat...
Processing for    heavy with X = first_board_lon and Y = first_board_lat...
Processing for commuter with X = first_board_lon and Y = first_board_lat...
Processing for    local with X = last_alight_lon and Y = last_alight_lat...
Processing for  express with X = last_alight_lon and Y = last_alight_lat...
Processing for    light with X = last_alight_lon and Y = last_alight_lat...
Processing for    ferry with X = last_alight_lon and Y = last_alight_lat...
Processing for    heavy with X = last_alight_lon and Y = last_alight_lat...
Processing for commuter with X = last_alight_lon and Y = last_alight_lat...

2017-12-27 11:52:27.770000


#### Combine data frames

In [9]:
df_obs_board  = (((df_obs_board_local.append(df_obs_board_express)).append(df_obs_board_ferry)).append(df_obs_board_commuter)).append(df_obs_board_heavy)
df_obs_board.rename(columns = {'tap': 'board_tap', 'tap_dist': 'board_tap_dist'}, inplace = True)
df_obs_board = df_obs_board.drop(['first_board_tech'],1)

df_obs_alight = (((df_obs_alight_local.append(df_obs_alight_express)).append(df_obs_alight_ferry)).append(df_obs_alight_commuter)).append(df_obs_alight_heavy)
df_obs_alight.rename(columns = {'tap': 'alight_tap', 'tap_dist': 'alight_tap_dist'}, inplace = True)
df_obs_alight = df_obs_alight.drop(['last_alight_tech'],1)

df_obs_board_alight = df_obs_board.join(df_obs_alight, how = 'outer')

#### Write out results

In [10]:
filename_out = 'M:/Data/OnBoard/Data and Reports/_geocoding Standardized/boarding_alighting_places_geocoded.csv'
df_obs_board_alight.to_csv(filename_out)

### Diagnostics

In [11]:
df_obs_board_alight

,first_board_lat,first_board_lon,board_tap,board_tap_dist,last_alight_lat,last_alight_lon,alight_tap,alight_tap_dist
Unique_ID,,,,,,,,
1---Caltrain---2014,37.776541,-122.395406,90742.0,0.000789,37.443340,-122.164182,291288.0,0.000078
1---SF Muni---2017,37.664238,-122.443710,190771.0,0.000751,37.792529,-122.434676,90139.0,0.000900
10---Caltrain---2014,37.580246,-122.345145,190773.0,0.000201,37.454382,-122.182266,190776.0,0.000138
10---SF Muni---2017,37.719953,-122.459290,90189.0,0.000204,37.792529,-122.434676,90139.0,0.000900
100---Caltrain---2014,37.378613,-122.030683,291292.0,0.000365,37.757692,-122.392318,90740.0,0.000180
100---SF Muni---2017,37.753067,-122.406210,90368.0,0.000970,37.754705,-122.396572,90112.0,0.000214
1000---BART---2015,37.752463,-122.418157,90733.0,0.000367,37.615910,-122.392721,190770.0,0.001811
1000---Caltrain---2014,37.443340,-122.164182,291288.0,0.000078,37.775206,-122.419208,90279.0,0.002293
1000---SF Muni---2017,37.765125,-122.419668,90490.0,0.000094,37.785850,-122.411980,90127.0,0.001171


In [12]:
df_obs_board_local.max()['tap_dist']

244.0613633585232

In [13]:
df_obs_board_express.max()['tap_dist']

0.4066065675973778

In [14]:
df_obs_board_ferry.max()['tap_dist']

0.04153917497886518

In [15]:
df_obs_board_commuter.max()['tap_dist']

0.6538858810835533

In [16]:
df_obs_board_light.max()['tap_dist']

159.69250772859033

In [17]:
df_obs_board_heavy.max()['tap_dist']

0.047687568744866446

In [18]:
df_obs_alight_local.max()['tap_dist']

39.54607360036195

In [19]:
df_obs_alight_express.max()['tap_dist']

0.4660029636859579

In [20]:
df_obs_alight_light.max()['tap_dist']

1.8393283075941236

In [21]:
df_obs_alight_ferry.max()['tap_dist']

0.018805161341113544

In [22]:
df_obs_alight_heavy.max()['tap_dist']

0.5966478151719906

In [23]:
df_obs_alight_commuter.max()['tap_dist']

1.1311503054962397